# 📌 ADIA Lab Structural Break Challenge  

**Assalam o Alaikum👋**  
In this notebook, we are going to explore the concept of **structural breaks (regime shifts)** – basically jab data ka trend, mean, or variance suddenly change ho jaye 📊.  

Structural break detection is an important problem because real-world data kabhi bhi smooth aur stable nahi hota. Kabhi kabhi beech me major shifts aate hain jo forecasting aur analysis dono ko effect kar dete hain.  

---

## 🔎 Challenge Overview  

Welcome to the **ADIA Lab Structural Break Challenge!**  
In this competition, you will analyze **univariate time series data** to determine whether a **structural break** has occurred at a specified boundary point.  

### 📖 What is a Structural Break?  

A **structural break** occurs when the process governing the data generation changes at a certain point in time.  
These changes can be subtle or dramatic, and detecting them accurately is crucial across domains:  

- 🌦 **Climatology** → shifts in long-term weather patterns  
- 🏭 **Industrial Monitoring** → detecting sudden machine behavior changes  
- 💹 **Finance** → market crashes or regime shifts  
- 🏥 **Healthcare** → sudden change in patient health indicators  

![Structural Break Example](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/competitions/structural-break/quickstarters/baseline/images/example.png)  

---

## 📝 Our Task  

For each time series in the **test set**, we need to predict a **score between `0` and `1`:**  

- `0` → No structural break at the specified boundary point  
- `1` → A structural break **did occur**  

---

## 📊 Evaluation Metric  

The challenge uses **ROC AUC (Area Under the Receiver Operating Characteristic Curve)** as the evaluation metric:  

- **ROC AUC ≈ 0.5** → No better than random guessing  
- **ROC AUC → 1.0** → Perfect detection performance  

More about ROC AUC: [sklearn.metrics.roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)  

---

## 📂 Notebook Flow 🚀  

1. **Exploratory Data Analysis (EDA)** – visualize aur samajhenge dataset.  
2. **Methods** – different techniques (statistical + ML-based) try karenge for break detection.  
3. **Evaluation** – compare karenge results aur dekhenge kaun sa method best perform karta hai.  

---

⚡ **Goal**: A clean, reproducible, and easy-to-follow Kaggle-style notebook – jahan beginner bhi seekh le aur advanced banda bhi enjoy kare.  

**Chalo shuru karte hain 🚀**  


In [ ]:
# import Important Libraries

!pip install antropy --quiet
!pip install PyWavelets --quiet


In [ ]:
import os
import typing

# Import your dependencies
import joblib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import sklearn.metrics
from scipy.stats import wasserstein_distance  # 1D Earth Mover's Distance
from scipy.stats import skew, kurtosis, ks_2samp
from scipy.stats import wasserstein_distance

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report, RocCurveDisplay
from scipy.stats import skew, kurtosis, ks_2samp, mannwhitneyu, wasserstein_distance
from scipy.signal import welch, hilbert
from statsmodels.tsa.stattools import acf, pacf

from scipy.signal import welch
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

### Crunch CLI SETUP

In [ ]:
### CRunch CLI Setup
%pip install crunch-cli --upgrade --quiet --progress-bar off
!crunch setup-notebook structural-break xm5KtLMSrYthCM1fOS5CsfpQ


import crunch

# Load the Crunch Toolings
crunch = crunch.load_notebook()

# Load the data simply
X_train, y_train, X_test = crunch.load_data()

### BASIC EDA

In [ ]:
# --- Dataset Overview ---
print("📂 Total training subsequences:", len(X_train))
print(X_train.shape)
print("📂 Total test subsequences:", len(X_test))
print("📂 y_train length:", len(y_train))

# --- Target Labels ---
print("\n📊 y_train distribution:")
print(y_train.value_counts(normalize=True))
print(y_train.value_counts())

# --- Check Nulls in y_train ---
print("\n📉 Missing values in y_train:", y_train.isnull().sum())

# --- Test set EDA (corrected) ---
print("\n📂 Test set overview:")
print("Number of test series:", len(X_test))
print("Shape of first 5 test series:", [np.array(df).shape for df in X_test[:5]])

# --- Nulls in X_train (safe) ---
train_nulls = 0
for x in X_train[:1000]:  # sample 1000 series
    try:
        arr = np.array(x, dtype=float)
        train_nulls += np.isnan(arr).sum()
    except:
        # non-numeric series
        train_nulls += 0

print(f"\n📉 Null values in first 1000 training series: {train_nulls}")

# --- Nulls in X_test (safe) ---
test_nulls = 0
for x in X_test:
    try:
        arr = np.array(x, dtype=float)
        test_nulls += np.isnan(arr).sum()
    except:
        test_nulls += 0

print(f"📉 Null values in test series: {test_nulls}")

# --- Example lengths of sequences ---
train_lengths = [len(x) for x in X_train[:1000]]  # sample
print(f"\n📏 Training series length stats (sample 1000):")
print("Number of datasets:", len(X_test))
print(pd.Series(train_lengths).describe())

## 📝 Basic EDA Notes (Updated)

- **Total Training Subsequences:** 23,715,734  
- **Total Test Subsequences:** 101  
- **y_train Length:** 10,001  

### 📊 y_train Distribution
- **Proportion:**  
  - False: 70.9%  
  - True: 29.1%  
- **Counts:**  
  - False: 7,092  
  - True: 2,909  
- Label imbalance exists → metric ROC AUC is robust to this.

### 📉 Missing Values
- **y_train:** 0  
- **X_train (first 1000 series):** 0  
- **X_test:** 0  
- No missing data → preprocessing easier.

### 📂 Test Set Overview
- Number of test series: 101  
- Shape of first 5 test series: `(2779,2), (1966,2), (1775,2), (3296,2), (1995,2)`  
- Each test series has variable length but 2 columns (probably period & value).

### 📏 Training Series Lengths (Sample 1000)
- Min: 5, Max: 6, Mean: 5.5  
- Mostly very short sequences → statistical/ML models will work on small windows.

**Notes:**  
- Dataset has **very large number of training subsequences**  
- No missing values → safe to use directly  
- Label imbalance is moderate → consider for evaluation metric  
- Next steps: visualize sample series, inspect True vs False structural breaks.


### Visualization of series

In [ ]:
# visualize a single time series
def plot_series(series_id):
    df = X_train.loc[series_id]
    plt.figure(figsize=(12,4))
    plt.plot(df.index, df["value"], label="value")
    plt.axvline(x=df[df["period"]==0].index.max(), color="red", linestyle="--", label="boundary")
    plt.title(f"Series ID {series_id} | Label: {y_train.loc[series_id]}")
    plt.legend()
    plt.show()

plot_series(0)
plot_series(1)


### Length of series

In [ ]:
#Length of series
series_lengths = X_train.groupby("id").size()
display(series_lengths.describe())

### Mean aur Std deviation comparison before and after the structural break:


In [ ]:
# Get indices
true_idx = y_train[y_train==True].index[0]
false_idx = y_train[y_train==False].index[0]

def compare_stats_plot(series_id):
    df = X_train.loc[series_id]  # assuming period & value columns
    before = df[df["period"]==0]["value"]
    after  = df[df["period"]==1]["value"]
    
    print(f"Series {series_id} | Label: {y_train.loc[series_id]}")
    print("Before mean:", before.mean(), " | std:", before.std())
    print("After  mean:", after.mean(), " | std:", after.std())
    
    # Plot
    plt.figure(figsize=(10,4))
    plt.plot(before.values, marker='o', label='Before Break')
    plt.plot(after.values, marker='o', label='After Break')
    plt.title(f"Series {series_id} | Label: {y_train.loc[series_id]}")
    plt.xlabel("Time Steps")
    plt.ylabel("Value")
    plt.legend()
    plt.show()

# Compare True series
compare_stats_plot(true_idx)

# Compare False series
compare_stats_plot(false_idx)


## 🔍 Structural Break Interpretation

**Series 0 | Label: False**  
- Mean before: 1.46e-5 | after: 6.37e-6  
- Std before: 0.00699 | after: 0.00688  
- Observation: Mean and std are almost the same → **No significant change**  
- Conclusion: **No structural break**  

**Series 2 | Label: True**  
- Mean before: 0.000389 | after: 0.00179  
- Std before: 0.0172 | after: 0.0229  
- Observation: Both mean and std increased → **Noticeable change in distribution**  
- Conclusion: **Structural break exists**  

**Summary:**  
- **False series:** stable values, no break  
- **True series:** sudden change in mean/variance → structural break


### Compariseon true and fase labels

In [ ]:
# Compariseon true and fase labels
def compare_true_false(X_train, y_train, true_id=None, false_id=None):
    # agar ID pass na ki ho to first True/False pick karo
    if true_id is None:
        true_id = y_train[y_train == True].index[0]
    if false_id is None:
        false_id = y_train[y_train == False].index[0]
    
    for series_id in [true_id, false_id]:
        df = X_train.loc[series_id]
        before = df[df["period"]==0]["value"]
        after  = df[df["period"]==1]["value"]
        
        fig, axes = plt.subplots(1, 2, figsize=(14,4))
        
        # time series plot
        axes[0].plot(df.index, df["value"], alpha=0.7)
        boundary = df[df["period"]==0].index.max()
        axes[0].axvline(boundary, color="red", linestyle="--", label="boundary")
        axes[0].set_title(f"Series {series_id} | Label: {y_train.loc[series_id]}")
        axes[0].legend()
        
        # histogram compare
        axes[1].hist(before, bins=50, alpha=0.5, label="before")
        axes[1].hist(after, bins=50, alpha=0.5, label="after")
        axes[1].set_title("Distribution before vs after")
        axes[1].legend()
        
        plt.suptitle(f"Stats → Before mean={before.mean():.4f}, std={before.std():.4f} | After mean={after.mean():.4f}, std={after.std():.4f}", fontsize=10)
        plt.show()

# run example
compare_true_false(X_train, y_train)



In [ ]:
X_test[0]

## 🔹 Advanced Feature Engineering Strategy

Our goal is to transform raw time series data into **informative features** that capture both statistical properties and structural changes between "before" and "after" segments. This enables machine learning models to detect subtle patterns indicative of structural breaks or anomalies.

---

### 1️⃣ Segmentation
- Each time series is divided into two segments based on the `period` column:
  - **Before (period=0)**
  - **After  (period=1)**
- Null values are dropped to ensure robust calculations.
- Short segments (<8 samples) are skipped to maintain reliability.

---

### 2️⃣ Basic Statistical Features
For both segments, compute:
- **Mean & Median** → central tendency
- **Standard Deviation (std)** → spread
- **Interquartile Range (IQR)** → variability

**Differences & Ratios:**
- Mean difference (`mean_diff = after_mean - before_mean`)
- Median difference (`median_diff`)
- Std difference & ratio (`std_diff`, `std_ratio`)
- IQR difference (`iqr_diff`)

These capture shifts in distribution after the structural break.

---

### 3️⃣ Complexity & Entropy Features
- **Permutation Entropy (PE)** → measure randomness/complexity of the signal
- **Multiscale Sample Entropy (MSE)** → complexity over multiple scales
- Compute differences between `after` and `before`:
  - `pe_m3_t1_diff`, `pe_m5_t1_diff`, `mse_diff`

---

### 4️⃣ Distance & Statistical Tests
- **Wasserstein Distance (EMD)** → distributional shift
- **Kolmogorov-Smirnov Statistic (KS)** → statistical difference between distributions
- **Mann-Whitney U Test p-value** → non-parametric comparison

---

### 5️⃣ Autocorrelation & Frequency Features
- **ACF Energy Difference** → captures change in temporal correlation
- **Spectral Features**:
  - Spectral Entropy difference → disorder in frequency domain
  - Spectral Centroid difference → shift in frequency “center of mass”
- **Wavelet Energy Difference** → change in localized energy across scales
- **Hilbert Instantaneous Frequency Difference** → change in dominant instantaneous frequency

---

### 6️⃣ Compression & Complexity
- **Lempel-Ziv Complexity (LZ)** difference → change in signal compressibility

---

### 7️⃣ Robustness
- All features handle missing values and small sequences safely.
- Infinite values replaced with 0.
- Optional libraries like `antropy` and `pywt` used if available, with safe fallbacks.

---

### ✅ Key Insight
- This approach captures **both statistical and structural changes** in time series.
- Combines **distributional, spectral, temporal, and complexity-based features**.
- Creates a rich feature matrix suitable for **machine learning models** like Logistic Regression, Random Forest, or XGBoost.



### 🔥 Feature Pipeline Structure (According to SHAP and correlation analysis ) in previous notebook

In [30]:
from scipy.stats import skew, kurtosis, ks_2samp, wasserstein_distance, linregress
from scipy.signal import hilbert, welch
from statsmodels.tsa.stattools import acf
import numpy as np
import pandas as pd
import antropy as ant
import pywt

# ---- Helper Functions ----
def robust_stats(arr):
    return {
        'mean': float(np.nanmean(arr)),
        'median': float(np.nanmedian(arr)),
        'std': float(np.nanstd(arr)),
        'iqr': float(np.nanpercentile(arr, 75) - np.nanpercentile(arr, 25))
    }

def slope_feature(x):
    try:
        slope, _, _, _, _ = linregress(range(len(x)), x)
        return float(slope)
    except:
        return np.nan

def fft_features(x):
    try:
        fft_vals = np.abs(np.fft.rfft(x))
        if len(fft_vals) == 0:
            return np.nan, np.nan, np.nan
        return float(np.max(fft_vals)), float(np.mean(fft_vals)), int(np.argmax(fft_vals))
    except:
        return np.nan, np.nan, np.nan

def spectral_features(x, fs=1.0):
    nperseg = min(256, max(8, len(x)))
    f, Pxx = welch(x, fs=fs, nperseg=nperseg)
    try:
        se = float(ant.spectral_entropy(x, sf=fs, method='welch', normalize=True))
    except:
        P = Pxx / (Pxx.sum() + 1e-12)
        se = float(-np.sum(P * np.log(P + 1e-12)))
    centroid = float(np.sum(f * Pxx) / (np.sum(Pxx) + 1e-12))
    return se, centroid

def hilbert_features(x):
    try:
        analytic = hilbert(x)
        amp_env = np.abs(analytic)
        return float(np.mean(amp_env)), float(np.std(amp_env))
    except:
        return np.nan, np.nan

def compute_permutation_entropy(x, m=3, tau=1):
    try:
        return float(ant.perm_entropy(x, order=m, delay=tau, normalize=True))
    except:
        return np.nan

def compute_multiscale_entropy(x, scales=(2,3,5)):
    vals = []
    for s in scales:
        xs = x[::s]
        if len(xs) > 10:
            try:
                vals.append(float(ant.sample_entropy(xs)))
            except:
                pass
    return float(np.mean(vals)) if vals else np.nan

def lz_complexity(x):
    try:
        sig = np.sign(x - np.nanmean(x)).astype(int)
        return float(ant.lziv_complexity(sig))
    except:
        return np.nan

# ---- Main Feature Builder ----# -

def build_selected_features(X_df, y_series):
    rows, ids = [], []

    for sid, df in X_df.groupby(level='id', sort=False):
        df = df.dropna(subset=['value'])
        before = df.loc[df['period']==0, 'value'].to_numpy(dtype=float)
        after  = df.loc[df['period']==1, 'value'].to_numpy(dtype=float)

        if len(before) < 8 or len(after) < 8:
            continue

        b = robust_stats(before)
        a = robust_stats(after)

        feats = {
            # basic stats
            'b_mean': b['mean'], 'a_mean': a['mean'],
            'b_median': b['median'], 'a_median': a['median'],
            'b_std': b['std'], 'a_std': a['std'],
            'b_iqr': b['iqr'], 'a_iqr': a['iqr'],

            # diffs/ratios
            'mean_diff': a['mean'] - b['mean'],
            'median_diff': a['median'] - b['median'],
            'iqr_diff': a['iqr'] - b['iqr'],
            'std_ratio': (a['std']+1e-9)/(b['std']+1e-9),
        }

        # shape
        feats['skew_diff']  = skew(after) - skew(before)
        feats['kurt_diff']  = kurtosis(after) - kurtosis(before)

        # slope/trend
        feats['slope_diff'] = slope_feature(after) - slope_feature(before)

        # fft/spectral
        max_b, _, peak_b = fft_features(before)
        max_a, _, peak_a = fft_features(after)
        feats['fft_peak_diff'] = max_a - max_b
        feats['fft_domfreq_shift'] = peak_a - peak_b

        se_b, cent_b = spectral_features(before)
        se_a, cent_a = spectral_features(after)
        feats['spectral_entropy_diff'] = se_a - se_b
        feats['spectral_centroid_diff'] = cent_a - cent_b

        # acf
        for lag in [1,2,5,10]:
            try:
                feats[f'acf_lag{lag}_diff'] = acf(after, nlags=lag, fft=True)[lag] - acf(before, nlags=lag, fft=True)[lag]
            except:
                feats[f'acf_lag{lag}_diff'] = np.nan

        # entropy family
        feats['perm_entropy_diff'] = compute_permutation_entropy(after) - compute_permutation_entropy(before)
        feats['ms_entropy_diff']   = compute_multiscale_entropy(after) - compute_multiscale_entropy(before)
        feats['lz_complexity_diff'] = lz_complexity(after) - lz_complexity(before)

        # distances/tests
        try:
            feats['ks_stat'] = ks_2samp(before, after).statistic
        except:
            feats['ks_stat'] = np.nan
        try:
            feats['wasserstein'] = wasserstein_distance(before, after)
        except:
            feats['wasserstein'] = np.nan

        # hilbert mean only
        hb_mean_b, _ = hilbert_features(before)
        hb_mean_a, _ = hilbert_features(after)
        feats['hilbert_mean_diff'] = hb_mean_a - hb_mean_b

        rows.append(feats)
        ids.append(sid)

    X_features = pd.DataFrame(rows, index=ids)
    X_features.index.name = 'id'
    X_features = X_features.replace([np.inf, -np.inf], np.nan).fillna(0)
    y_features = y_series.loc[X_features.index].astype(int)
    return X_features, y_features


# ---- Single-series extractor ----
def extract_seleted_features_series(df):
    df = df.dropna(subset=['value'])
    before = df.loc[df['period']==0, 'value'].to_numpy(dtype=float)
    after  = df.loc[df['period']==1, 'value'].to_numpy(dtype=float)

    if len(before) < 8 or len(after) < 8:
        return None

    b = robust_stats(before)
    a = robust_stats(after)

    feats = {
        # basic stats
        'b_mean': b['mean'], 'a_mean': a['mean'],
        'b_median': b['median'], 'a_median': a['median'],
        'b_std': b['std'], 'a_std': a['std'],
        'b_iqr': b['iqr'], 'a_iqr': a['iqr'],

        # diffs/ratios
        'mean_diff': a['mean'] - b['mean'],
        'median_diff': a['median'] - b['median'],
        'iqr_diff': a['iqr'] - b['iqr'],
        'std_ratio': (a['std']+1e-9)/(b['std']+1e-9),
    }

    # shape
    feats['skew_diff']  = skew(after) - skew(before)
    feats['kurt_diff']  = kurtosis(after) - kurtosis(before)

    # slope/trend
    feats['slope_diff'] = slope_feature(after) - slope_feature(before)

    # fft/spectral
    max_b, _, peak_b = fft_features(before)
    max_a, _, peak_a = fft_features(after)
    feats['fft_peak_diff'] = max_a - max_b
    feats['fft_domfreq_shift'] = peak_a - peak_b

    se_b, cent_b = spectral_features(before)
    se_a, cent_a = spectral_features(after)
    feats['spectral_entropy_diff'] = se_a - se_b
    feats['spectral_centroid_diff'] = cent_a - cent_b

    # acf
    for lag in [1,2,5,10]:
        try:
            feats[f'acf_lag{lag}_diff'] = acf(after, nlags=lag, fft=True)[lag] - acf(before, nlags=lag, fft=True)[lag]
        except:
            feats[f'acf_lag{lag}_diff'] = np.nan

    # entropy family
    feats['perm_entropy_diff'] = compute_permutation_entropy(after) - compute_permutation_entropy(before)
    feats['ms_entropy_diff']   = compute_multiscale_entropy(after) - compute_multiscale_entropy(before)
    feats['lz_complexity_diff'] = lz_complexity(after) - lz_complexity(before)

    # distances/tests
    try:
        feats['ks_stat'] = ks_2samp(before, after).statistic
    except:
        feats['ks_stat'] = np.nan
    try:
        feats['wasserstein'] = wasserstein_distance(before, after)
    except:
        feats['wasserstein'] = np.nan

    # hilbert mean only
    hb_mean_b, _ = hilbert_features(before)
    hb_mean_a, _ = hilbert_features(after)
    feats['hilbert_mean_diff'] = hb_mean_a - hb_mean_b

    return feats


In [31]:
# --- Apply feature engineering to check the impact ---
X_features, y_features = build_selected_features(X_train, y_train)
print(X_features.shape)
print(y_features.value_counts())

(10001, 29)
structural_breakpoint
0    7092
1    2909
Name: count, dtype: int64


In [32]:
X_features.sample()

,b_mean,a_mean,b_median,a_median,b_std,a_std,b_iqr,a_iqr,mean_diff,median_diff,...,acf_lag1_diff,acf_lag2_diff,acf_lag5_diff,acf_lag10_diff,perm_entropy_diff,ms_entropy_diff,lz_complexity_diff,ks_stat,wasserstein,hilbert_mean_diff
id,,,,,,,,,,,,,,,,,,,,,
731,0.001303,0.00086,0.000843,0.002584,0.0144,0.016119,0.015806,0.019506,-0.000442,0.001742,...,-0.012684,0.048913,0.023514,0.015169,0.00136,0.0,-41.0,0.07331,0.002299,0.002619


In [33]:
X_features.isna().sum().sort_values(ascending=False)


b_mean                    0
fft_peak_diff             0
wasserstein               0
ks_stat                   0
lz_complexity_diff        0
ms_entropy_diff           0
perm_entropy_diff         0
acf_lag10_diff            0
acf_lag5_diff             0
acf_lag2_diff             0
acf_lag1_diff             0
spectral_centroid_diff    0
spectral_entropy_diff     0
fft_domfreq_shift         0
slope_diff                0
a_mean                    0
kurt_diff                 0
skew_diff                 0
std_ratio                 0
iqr_diff                  0
median_diff               0
mean_diff                 0
a_iqr                     0
b_iqr                     0
a_std                     0
b_std                     0
a_median                  0
b_median                  0
hilbert_mean_diff         0
dtype: int64

### Model Training And Evaluation

In [ ]:
%%writefile main.py



In [ ]:

# ======= Refactor-friendly train() and infer() =======
import os
import joblib
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_sample_weight
import matplotlib.pyplot as plt

def train(X_train: pd.DataFrame, y_train: pd.Series, model_directory_path: str):
    """
    Train stacking model with XGB as meta learner, 7-fold CV, full class imbalance handling.
    """
    os.makedirs(model_directory_path, exist_ok=True)
    rng_seed = 42

    # 1) Feature extraction
    X_features, y_features = build_all_features(X_train, y_train)
    print(f"[train] Features built: {X_features.shape}, labels: {y_features.value_counts().to_dict()}")

    # 2) Compute positive class ratio for XGB
    neg = len(y_features) - int(y_features.sum())
    pos = int(y_features.sum())
    pos_ratio = neg / (pos + 1e-9)

    # 3) Define base models with full imbalance handling
    base_models = [
        ("xgb_base", xgb.XGBClassifier(
            n_estimators=400, learning_rate=0.05, max_depth=5,
            subsample=0.8, colsample_bytree=0.8,
            eval_metric="logloss", use_label_encoder=False,
            random_state=rng_seed, scale_pos_weight=pos_ratio
        )),
        ("rf", RandomForestClassifier(
            n_estimators=400, max_depth=12, n_jobs=-1,
            random_state=rng_seed, class_weight="balanced"
        )),
        ("hgb", HistGradientBoostingClassifier(
            max_iter=300, max_depth=5, learning_rate=0.05,
            random_state=rng_seed, class_weight="balanced"
        )),
        ("mlp", Pipeline([
            ("scaler", RobustScaler()),
            ("mlp", MLPClassifier(hidden_layer_sizes=(128,), max_iter=500, random_state=rng_seed))
        ])),
        ("logreg", Pipeline([
            ("scaler", RobustScaler()),
            ("logreg", LogisticRegression(solver="liblinear", class_weight="balanced", random_state=rng_seed))
        ]))
    ]

    # 4) Prepare 7-fold GroupKFold
    gkf = GroupKFold(n_splits=7)
    groups = X_features.index
    oof_preds_per_model = {name: np.zeros(len(X_features)) for name, _ in base_models}
    fold_results = pd.DataFrame(columns=[f"fold_{i}" for i in range(1, 8)], index=[name for name, _ in base_models])
    fold_results.index.name = "model"

    calibrated_estimators = []

    # 5) Fit base models with CV and calibration
    for name, model in base_models:
        print(f"[train] CV for base model: {name}")
        fold_preds = np.zeros(len(X_features))
        for fold, (tr_idx, val_idx) in enumerate(gkf.split(X_features, y_features, groups)):
            X_tr, X_val = X_features.iloc[tr_idx], X_features.iloc[val_idx]
            y_tr, y_val = y_features.iloc[tr_idx], y_features.iloc[val_idx]

            # Compute sample_weight for MLP
            if name == "mlp":
                sample_weight = compute_sample_weight(class_weight="balanced", y=y_tr)
                model.fit(X_tr, y_tr, mlp__sample_weight=sample_weight)
            else:
                model.fit(X_tr, y_tr)

            proba = model.predict_proba(X_val)[:, 1]
            fold_preds[val_idx] = proba

            # Record fold ROC-AUC
            fold_results.loc[name, f"fold_{fold+1}"] = roc_auc_score(y_val, proba)
            print(f"   fold {fold} | {name} ROC-AUC: {fold_results.loc[name, f'fold_{fold+1}']:.4f}")

        oof_score = roc_auc_score(y_features, fold_preds)
        print(f"[train] {name} OOF ROC-AUC: {oof_score:.4f}")
        oof_preds_per_model[name] = fold_preds.copy()

        # Fit calibrated model on full data
        calibrated = CalibratedClassifierCV(model, method="isotonic", cv=3)
        calibrated.fit(X_features, y_features)
        calibrated_estimators.append((name, calibrated))

    # 6) Meta learner = XGB with imbalance handling
    meta = xgb.XGBClassifier(
        n_estimators=300, learning_rate=0.05, max_depth=5,
        subsample=0.8, colsample_bytree=0.8,
        eval_metric="logloss", use_label_encoder=False,
        random_state=rng_seed, scale_pos_weight=pos_ratio
    )

    # 7) Stacking
    stack = StackingClassifier(
        estimators=calibrated_estimators,
        final_estimator=meta,
        stack_method="predict_proba",
        passthrough=True,
        cv=7,
        n_jobs=-1
    )

    # 8) Fit final stacking
    stack.fit(X_features, y_features)
    final_oof = roc_auc_score(y_features, stack.predict_proba(X_features)[:, 1])
    print(f"[train] Final stacking OOF ROC-AUC: {final_oof:.4f}")

    # 9) Add final stacking OOF to fold_results
    fold_results.loc["stacking_final", :] = [final_oof]*fold_results.shape[1]
    fold_results["mean_auc"] = fold_results.mean(axis=1)
    print("\n=== Fold ROC-AUC Results ===")
    print(fold_results)

    # 10) Plot ROC-AUC per fold
    plt.figure(figsize=(10,6))
    plot_df = fold_results.drop(columns="mean_auc")
    for model in plot_df.index:
        plt.plot(plot_df.columns, plot_df.loc[model], marker='o', label=model)
    plt.title("ROC-AUC per Fold for Base Models and Stacking")
    plt.xlabel("Fold")
    plt.ylabel("ROC-AUC")
    plt.ylim(0.5, 1.0)
    plt.grid(True)
    plt.legend(loc="lower right")
    plt.show()

    # 11) Save artifact
    artifact = {
        "feature_cols": list(X_features.columns),
        "stacking_model": stack,
        "oof_per_model": oof_preds_per_model,
        "train_oof_score": float(final_oof),
        "random_seed": rng_seed
    }
    joblib.dump(artifact, os.path.join(model_directory_path, "model.joblib"))
    print(f"[train] Saved artifact to {os.path.join(model_directory_path, 'model.joblib')}")

    
def infer(X_test, model_directory_path):
    """
    Inference function using trained stacking model.
    - X_test: MultiIndex DataFrame (id,time) with columns ['value','period']
    - model_directory_path: path to saved model.joblib
    Returns: DataFrame with 'id', 'pred_proba', 'pred_label'
    """

    import os, joblib, pandas as pd
    
    # Load artifact
    artifact = joblib.load(os.path.join(model_directory_path, "model.joblib"))
    stack_model = artifact["stacking_model"]
    feature_cols = artifact["feature_cols"]

    # signal ready
    yield

    if X_test is None:
        # safety check
        return

    # process each test DF (single series) and extract features then predict
    for df in X_test:
        feats = extract_selected_features_series(df)
        if feats is None:
            yield 0.0
            continue
        x = pd.DataFrame([feats])
        x = x.reindex(columns=feature_cols).fillna(0)
        proba = float(stack.predict_proba(x)[0, 1])
        yield proba




In [43]:
X_train, y_train, X_test = crunch.load_data()
train(X_train, y_train, "artifacts")

data/X_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match
[train] Features built: (10001, 29), labels: {0: 7092, 1: 2909}
[train] CV for base model: xgb_base
   fold 0 | xgb_b

TypeError: BaseMultilayerPerceptron.fit() got an unexpected keyword argument 'sample_weight'

In [ ]:
import matplotlib.pyplot as plt

def plot_fold_roc(fold_results):
    plt.figure(figsize=(10,6))

    # Exclude mean column
    plot_df = fold_results.drop(columns="mean_auc")

    for model in plot_df.index:
        plt.plot(plot_df.columns, plot_df.loc[model], marker='o', label=model)

    plt.title("ROC-AUC per Fold for Base Models and Stacking")
    plt.xlabel("Fold")
    plt.ylabel("ROC-AUC")
    plt.ylim(0.5, 1.0)
    plt.grid(True)
    plt.legend(loc="lower right")
    plt.show()


# After printing fold_results
plot_fold_roc(fold_results)


In [ ]:
preds = list(infer(X_test, "artifacts"))
print(preds[:10])


In [ ]:
crunch.test(
    # Uncomment to disable the train
    #force_first_train=False,

    # Uncomment to disable the determinism check
    # no_determinism_check=True,
)



In [ ]:
prediction = pd.read_parquet("data/prediction.parquet")

prediction

In [ ]:
# Load the targets
target = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"]

# Call the scoring function
sklearn.metrics.roc_auc_score(
    target,
    prediction,
)

In [ ]:
target